<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# AI in Finance

**Workshop at Texas State University (October 2023)**

**_Simulating Financial Time Series Data_**

Dr. Yves J. Hilpisch | The Python Quants GmbH | http://tpq.io

## Generated Data

### Noisy Data

In [ ]:
class ActionSpace:
    def sample(self):
        return random.randint(0, 1)

In [ ]:
import numpy as np
import pandas as pd
from numpy.random import default_rng

In [ ]:
rng = default_rng()

In [ ]:
class NoisyData:
    url = 'https://certificate.tpq.io/eod_prices.csv'
    def __init__(self, symbol, feature, n_features=4,
                 min_accuracy=0.485, noise=True,
                 noise_std=0.001):
        self.symbol = symbol
        self.feature = feature
        self.n_features = n_features
        self.noise = noise
        self.noise_std = noise_std
        self.action_space = ActionSpace()
        self.min_accuracy = min_accuracy
        self._get_data()
        self._prepare_data()
    def _get_data(self):
        self.raw = pd.read_csv(self.url,
                index_col=0, parse_dates=True)

In [ ]:
class NoisyData(NoisyData):
    def _prepare_data(self):
        self.data = pd.DataFrame(self.raw[self.symbol]).dropna()
        if self.noise:
            std = self.data.mean() * self.noise_std
            self.data[self.symbol] = (self.data[self.symbol] +
                rng.normal(0, std, len(self.data)))
        self.data['r'] = np.log(self.data / self.data.shift(1))
        self.data['d'] = np.where(self.data['r'] > 0, 1, 0)
        self.data.dropna(inplace=True)
        mi, ma = self.data.max(), self.data.min()
        self.data_ = (self.data - mi) / (ma - mi)
    def reset(self):
        if self.noise:
            self._prepare_data()
        self.bar = self.n_features
        self.treward = 0
        state = self.data_[self.feature].iloc[
            self.bar - self.n_features:self.bar].values
        return state, {}

In [ ]:
class NoisyData(NoisyData):
    def step(self, action):
        if action == self.data['d'].iloc[self.bar]:
            correct = True
        else:
            correct = False
        reward = 1 if correct else 0 
        self.treward += reward
        self.bar += 1
        self.accuracy = self.treward / (self.bar - self.n_features)
        if self.bar >= len(self.data):
            done = True
        elif reward == 1:
            done = False
        elif (self.accuracy < self.min_accuracy and
              self.bar > self.n_features + 15):
            done = True
        else:
            done = False
        next_state = self.data_[self.feature].iloc[
            self.bar - self.n_features:self.bar].values
        return next_state, reward, done, False, {}

In [ ]:
fin = NoisyData(symbol='AAPL', feature='AAPL',
                noise=True, noise_std=0.005)

In [ ]:
fin.reset()

In [ ]:
fin.reset()

In [ ]:
fin = NoisyData('AAPL', 'r', n_features=4,
                noise=True, noise_std=0.025)

In [ ]:
fin.reset()

In [ ]:
fin.reset()

In [ ]:
from pylab import plt, mpl
plt.style.use('seaborn-v0_8')
mpl.rcParams['figure.dpi'] = 300
mpl.rcParams['savefig.dpi'] = 300
mpl.rcParams['font.family'] = 'serif'

In [ ]:
for _ in range(5):
    fin.reset()
    fin.data[fin.symbol].loc['2023-7-1':].plot(lw=1)

In [ ]:
for _ in range(5):
    fin.reset()
    fin.data[fin.symbol].plot(lw=1, alpha=0.5)

### Simulated Data

In [ ]:
class Simulation:
    def __init__(self, symbol, feature, n_features,
                 start, end, periods,
                 min_accuracy=0.525, x0=100,
                 kappa=1, theta=100, sigma=0.2,
                 normalize=True, new=False):
        self.symbol = symbol
        self.feature = feature
        self.n_features = n_features
        self.start = start
        self.end = end
        self.periods = periods
        self.x0 = x0
        self.kappa = kappa
        self.theta = theta
        self.sigma = sigma
        self.min_accuracy = min_accuracy
        self.normalize = normalize
        self.new = new
        self.action_space = ActionSpace()
        self._simulate_data()
        self._prepare_data()

In [ ]:
import math
import random
class Simulation(Simulation):
    def _simulate_data(self):
        index = pd.date_range(start=self.start,
                    end=self.end, periods=self.periods)
        s = [self.x0]
        dt = (index[-1] - index[0]).days / 365 / self.periods
        for t in range(1, len(index)):
            s_ = (s[t - 1] + self.kappa * (self.theta - s[t - 1]) * dt +
                  s[t - 1] * self.sigma * math.sqrt(dt) * random.gauss(0, 1))
            s.append(s_)
        
        self.data = pd.DataFrame(s, columns=[self.symbol], index=index)

In [ ]:
class Simulation(Simulation):
    def _prepare_data(self):
        self.data['r'] = np.log(self.data / self.data.shift(1))
        self.data.dropna(inplace=True)
        if self.normalize:
            self.mu = self.data.mean()
            self.std = self.data.std()
            self.data_ = (self.data - self.mu) / self.std
        else:
            self.data_ = self.data.copy()
        self.data['d'] = np.where(self.data['r'] > 0, 1, 0)
        self.data['d'] = self.data['d'].astype(int)

In [ ]:
class Simulation(Simulation):
    def _get_state(self):
        return self.data_[self.feature].iloc[self.bar -
                                self.n_features:self.bar]
    #def seed(self, seed):
    #    random.seed(seed)
    #    np.random.seed(seed)
    #    tf.random.set_random_seed(seed)
    def reset(self):
        self.treward = 0
        self.accuracy = 0
        self.bar = self.n_features
        if self.new:
            self._simulate_data()
            self._prepare_data()
        state = self._get_state()
        return state.values, {}

In [ ]:
class Simulation(Simulation):
    def step(self, action):
        if action == self.data['d'].iloc[self.bar]:
            correct = True
        else:
            correct = False
        reward = 1 if correct else 0 
        self.treward += reward
        self.bar += 1
        self.accuracy = self.treward / (self.bar - self.n_features)
        if self.bar >= len(self.data):
            done = True
        elif reward == 1:
            done = False
        elif (self.accuracy < self.min_accuracy and
              self.bar > self.n_features + 15):
            done = True
        else:
            done = False
        next_state = self.data_[self.feature].iloc[
            self.bar - self.n_features:self.bar].values
        return next_state, reward, done, False, {}

In [ ]:
sym = 'EUR='

In [ ]:
env_base = Simulation(sym, sym, 5, start='2023-1-1', end='2024-1-1',
                 periods=50, x0=1, kappa=1, theta=1.1, sigma=0.0,
                 normalize=True)

In [ ]:
env_trend = Simulation(sym, sym, 5, start='2023-1-1', end='2024-1-1',
                 periods=50, x0=1, kappa=1, theta=2, sigma=0.1,
                 normalize=True)

In [ ]:
env_mrev = Simulation(sym, sym, 5, start='2023-1-1', end='2024-1-1',
                 periods=50, x0=1, kappa=1, theta=1, sigma=0.1,
                 normalize=True)

In [ ]:
env_base.data[sym].plot(figsize=(10, 6),label='baseline')
env_trend.data[sym].plot(label='trend', style='--')
env_mrev.data[sym].plot(label='mean-reversion', style='-.')
plt.legend();

In [ ]:
sim = Simulation(sym, 'r', 4, start='2023-1-1', end='2028-1-1',
                 periods=500, min_accuracy=0.6, x0=1,
                 kappa=1.25, theta=2, sigma=0.15,
                 normalize=True, new=True)

In [ ]:
for _ in range(10):
    sim.reset()
    sim.data[sym].plot(figsize=(10, 6));

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:team@tpq.io">team@tpq.io</a>